In [5]:
import pandas as pd
import numpy as np
import keras
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
import random
import gc
import re
import MeCab
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


#df_account = pd.read_csv(work_path + account_path, sep = '\t', compression='gzip')
df_order = pd.read_csv(work_path + order_path, sep = '\t', compression='gzip')
#df_deal = pd.read_csv(work_path + deal_path, sep = '\t', compression='gzip')
#df_dealprofile = pd.read_csv(work_path + deal_profile_path, sep = '\t', compression='gzip')
#df_dealprofile2 = pd.read_csv(work_path + deal_profile_path2, sep = '\t', compression='gzip')

In [6]:
df_order = df_order[['account_id', 'deal_id', 'ins_tm']]
df_order['rating'] = 1
df_order.account_id = df_order.account_id.astype('category').cat.codes.values
df_order.deal_id = df_order.deal_id.astype('category').cat.codes.values

In [7]:
item_list = list(df_order.deal_id.unique())
user_list = list(df_order.account_id.unique())

num_epo = 0
Negative_sample = []
for user in user_list:
    bought_list = list(df_order[df_order.account_id == user].deal_id.values)
    negative_list = list(set(item_list)^set(bought_list))
    some = random.sample(negative_list, len(bought_list))
    for i in range(len(bought_list)):
        Negative_sample.append([user, some[i], 0])
    if num_epo%10000 == 0:
        print(str(num_epo) + '/' + str(len(user_list)) + 'finished!')
    num_epo += 1

0/341409finished!
10000/341409finished!
20000/341409finished!
30000/341409finished!
40000/341409finished!
50000/341409finished!
60000/341409finished!
70000/341409finished!
80000/341409finished!
90000/341409finished!
100000/341409finished!
110000/341409finished!
120000/341409finished!
130000/341409finished!
140000/341409finished!
150000/341409finished!
160000/341409finished!
170000/341409finished!
180000/341409finished!
190000/341409finished!
200000/341409finished!
210000/341409finished!
220000/341409finished!
230000/341409finished!
240000/341409finished!
250000/341409finished!
260000/341409finished!
270000/341409finished!
280000/341409finished!
290000/341409finished!
300000/341409finished!
310000/341409finished!
320000/341409finished!
330000/341409finished!
340000/341409finished!


In [10]:
df_Ns = pd.DataFrame(Negative_sample, columns=['account_id', 'deal_id', 'rating'])

In [ ]:
df_order = df_order.append(df_Ns, ignore_index=True)

In [ ]:
df_order.to_csv('df_order_add_negasmpl.gz', compression = 'gzip')

# simple RS

In [2]:
df_order = pd.read_csv('df_order_add_negasmpl.gz', compression='gzip')

In [3]:
df_order = df_order[['account_id', 'deal_id', 'rating']]

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_order, test_size = 0.1)

In [5]:
train.shape, test.shape

((3028861, 3), (336541, 3))

In [6]:
train, valid = train_test_split(train, test_size = 0.2)

In [7]:
train.shape, valid.shape, test.shape

((2423088, 3), (605773, 3), (336541, 3))

In [25]:
n_users, n_item = len(df_order.account_id.unique()), len(df_order.deal_id.unique())
n_latent_factors = 5 

In [26]:
item_input = keras.layers.Input(shape=[1], name='Item')
item_embedding = keras.layers.Embedding(n_item + 1, n_latent_factors, name='Item-Embedding')(item_input)
item_vec = keras.layers.Flatten(name='FlattenItem')(item_embedding)

user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))

dot = keras.layers.dot([item_vec, user_vec], -1, name='DotProduct')
prod = output = keras.layers.Dense(1, activation='sigmoid')(dot)
model = keras.Model([user_input, item_input], prod)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [27]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Item-Embedding (Embedding)      (None, 1, 5)         342295      Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 5)         1707050     User[0][0]                       
__________________________________________________________________________________________________
FlattenIte

In [28]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

check_point = ModelCheckpoint('model.hdf5', verbose=True, save_best_only = True)
early_stop = EarlyStopping(patience=5, verbose=True)
model.fit([train.account_id, train.deal_id], train.rating, 
          validation_data=([valid.account_id, valid.deal_id], valid.rating),
          batch_size=128,
          epochs=20, 
          verbose=True,
         callbacks=[early_stop, check_point])

Train on 2423088 samples, validate on 605773 samples
Epoch 1/20
2423088/2423088 [==============================] - 186s 77us/step - loss: 0.6190 - acc: 0.6458 - val_loss: 0.5150 - val_acc: 0.7510

Epoch 00001: val_loss improved from inf to 0.51500, saving model to model.hdf5
Epoch 2/20
2423088/2423088 [==============================] - 187s 77us/step - loss: 0.4076 - acc: 0.8236 - val_loss: 0.4743 - val_acc: 0.7798

Epoch 00002: val_loss improved from 0.51500 to 0.47432, saving model to model.hdf5
Epoch 3/20
2423088/2423088 [==============================] - 187s 77us/step - loss: 0.3374 - acc: 0.8609 - val_loss: 0.4718 - val_acc: 0.7831

Epoch 00003: val_loss improved from 0.47432 to 0.47179, saving model to model.hdf5
Epoch 4/20
2423088/2423088 [==============================] - 189s 78us/step - loss: 0.3041 - acc: 0.8757 - val_loss: 0.4784 - val_acc: 0.7824

Epoch 00004: val_loss did not improve from 0.47179
Epoch 5/20
2423088/2423088 [==============================] - 193s 80us/ste

In [29]:
model.load_weights('model.hdf5')

In [30]:
pred_result = model.predict([test.account_id, test.deal_id])

In [31]:
right_num = 0
for i in range(len(pred_result)):
    if int(pred_result[i][0] >=0.5) == test.rating.values[i]:
        right_num += 1
print('accuracy: {}'.format((right_num / len(pred_result))))

accuracy: 0.7833131772948912


# result  n_latent_factor  optimizer  loss  acc
## 3  adam  mse  0.7603412362832463
## 3  SGD    mse  acc: 0.50
## 3  adam  crossEntropy 0.7822672423270864 (sigmoid(pred)
## 5  adam  crpssEntropy 0.7833131772948912 (sigmoid(pred)